In [1]:
%load_ext autoreload
%autoreload 2

## Creating Audio File

Checkout https://beta.elevenlabs.io/history for full log of audio creation

In [2]:
file_name = "./kitten_full"
mp3_file_name = file_name + ".mp3"
wav_file_name = file_name + ".wav"

generate_new_audio = False
name_of_voice = "Bella"
text = "I love you"

In [3]:
# Get voice ID
import requests

r1 = requests.get('https://api.elevenlabs.io/v1/voices')
resp = r1.json()
rid = ""
for voice in resp["voices"]:
    if voice["name"] == name_of_voice:
        print(voice)
        rid = voice["voice_id"]
        break;
rid

{'voice_id': 'EXAVITQu4vr4xnSDxMaL', 'name': 'Bella', 'samples': None, 'category': 'premade', 'fine_tuning': {'is_allowed_to_fine_tune': False, 'fine_tuning_requested': False, 'finetuning_state': 'not_started', 'verification_attempts': None, 'verification_attempts_count': 0}, 'labels': {}, 'preview_url': 'https://storage.googleapis.com/eleven-public-prod/premade/voices/EXAVITQu4vr4xnSDxMaL/04365bce-98cc-4e99-9f10-56b60680cda9.mp3', 'available_for_tiers': [], 'settings': None}


'EXAVITQu4vr4xnSDxMaL'

In [4]:
# Get audio file, minimize runs cost money
import json

if generate_new_audio:
    url = "https://api.elevenlabs.io/v1/text-to-speech/{voice_id}".format(voice_id=rid)
    print(url)
    headers = {'xi-api-key': "aa42307ce6bfbefcfd2abd4d8634127b"}
    payload = {
      "text": text,
      "voice_settings": {
        "stability": 0.3,
        "similarity_boost": 0.75
      }
    }

    r2 = requests.post(url, data=json.dumps(payload), headers=headers)
    print(r2)

In [5]:
# Save and play audio file
import os

if generate_new_audio:
    with open(mp3_file_name, 'wb') as f:
        f.write(r2.content)

    os.system("mpg123 " + mp3_file_name)

In [6]:
# convert mp3 to wav                                                            
from pydub import AudioSegment
from pydub.playback import play

if generate_new_audio:
    sound = AudioSegment.from_mp3(mp3_file_name)
    sound.export(wav_file_name, format="wav")

    # test sound
    song = AudioSegment.from_wav(wav_file_name)
    play(song)

## Generate phonemes
- Uses https://github.com/m-bain/whisperX

In [7]:
# no extension, must be wav
import os
cwd = os.getcwd() #pwd
print(cwd)

audio_file_name = wav_file_name
base_audio_file_name = os.path.basename(audio_file_name)
print("File name:", audio_file_name)
print("Base name:", base_audio_file_name)

/home/paperspace/repos/infinitenews/ForcedAlignment
File name: ./kitten_full.wav
Base name: kitten_full.wav


In [8]:
import gc
import torch
import whisperx

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# transcribe with original whisper
model = whisperx.load_model("medium.en", device)
# model = whisperx.load_model("base.en", device)

In [9]:
result = model.transcribe(audio_file_name)
print(result["segments"]) # before alignment

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 1.36, 'text': ' Good evening, everyone.', 'tokens': [4599, 6180, 11, 2506, 13], 'temperature': 0.0, 'avg_logprob': -0.15060143077045407, 'compression_ratio': 1.6151079136690647, 'no_speech_prob': 0.00961030088365078}, {'id': 1, 'seek': 0, 'start': 1.36, 'end': 5.88, 'text': ' Sydney Scott here bringing you the latest news from the upside down world.', 'tokens': [11852, 4746, 994, 6079, 345, 262, 3452, 1705, 422, 262, 17196, 866, 995, 13], 'temperature': 0.0, 'avg_logprob': -0.15060143077045407, 'compression_ratio': 1.6151079136690647, 'no_speech_prob': 0.00961030088365078}, {'id': 2, 'seek': 0, 'start': 5.88, 'end': 12.44, 'text': ' It appears that Ponzi schemes, once thought to be a financial fraud tool used only by humans,', 'tokens': [632, 3568, 326, 350, 13569, 72, 16546, 11, 1752, 1807, 284, 307, 257, 3176, 7394, 2891, 973, 691, 416, 5384, 11], 'temperature': 0.0, 'avg_logprob': -0.15060143077045407, 'compression_ratio': 1.615107913669064

In [10]:
# load alignment model and metadata
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)

# align whisper output
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file_name, device)

# print(result_aligned["segments"]) # after alignment
result_aligned["word_segments"]

[{'text': 'Good', 'start': 0.10149253731343286, 'end': 0.24358208955223884},
 {'text': 'evening,', 'start': 0.30447761194029854, 'end': 0.5480597014925374},
 {'text': 'everyone.', 'start': 0.6089552238805971, 'end': 0.9946268656716419},
 {'text': 'Sydney', 'start': 1.5207111111111111, 'end': 1.8220444444444444},
 {'text': 'Scott', 'start': 1.8622222222222222, 'end': 2.203733333333333},
 {'text': 'here', 'start': 2.243911111111111, 'end': 2.424711111111111},
 {'text': 'bringing', 'start': 2.484977777777778, 'end': 2.786311111111111},
 {'text': 'you', 'start': 2.826488888888889, 'end': 2.947022222222222},
 {'text': 'the', 'start': 3.0072888888888887, 'end': 3.087644444444444},
 {'text': 'latest', 'start': 3.127822222222222, 'end': 3.449244444444444},
 {'text': 'news', 'start': 3.5295999999999994, 'end': 3.750577777777777},
 {'text': 'from', 'start': 3.8108444444444443, 'end': 3.9514666666666667},
 {'text': 'the', 'start': 3.971555555555555, 'end': 4.071999999999999},
 {'text': 'upside', 

In [11]:
test_words = ["that's", "Help! ", "multiverse", "cat", "permit", "surprise", ",whales"]

In [12]:
import nltk

try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

In [13]:
import re
from functools import lru_cache
from itertools import product as iterprod

def cleanWord(word):
    regex = r"[^\w\']"
    return re.sub(regex, "", word).lower()

def arpabetWithoutNumbers(word):
    list_of_phones = []
    for phones in arpabet[word]:
        new_phones = ["".join(filter(lambda x: x.isalpha(), phone)) for phone in phones]
        list_of_phones.append(new_phones)
    return list_of_phones

def wordbreak(s):
    s = cleanWord(s)
    if s in arpabet:
        return arpabetWithoutNumbers(s)
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabetWithoutNumbers(pre), wordbreak(suf))]
    return None

# Example words: 
for w in test_words:
    print(wordbreak(w))

[['DH', 'AE', 'T', 'S']]
[['HH', 'EH', 'L', 'P']]
[['M', 'AH', 'L', 'T', 'IY', 'V', 'ER', 'S']]
[['K', 'AE', 'T']]
[['P', 'ER', 'M', 'IH', 'T'], ['P', 'ER', 'M', 'IH', 'T']]
[['S', 'ER', 'P', 'R', 'AY', 'Z'], ['S', 'AH', 'P', 'R', 'AY', 'Z']]
[['W', 'EY', 'L', 'Z'], ['HH', 'W', 'EY', 'L', 'Z']]


In [63]:
def letterToPhone(word, phones):
    # print("LTP", word, phones)
    if word == "":
        return []

    for i in reversed(range(1, len(word))):
        for j in range(0, len(word) - i + 1):
            subword = word[j:i+j]
            cur_possible_phones = wordbreak(subword)
            #For when has contractions
            if not cur_possible_phones:
#                 print("Skipping", i, j, subword, cur_possible_phones)
                continue
            else:
                cur_possible_phones = cur_possible_phones + [[subword.upper()]]
#                 print(i, j, subword, cur_possible_phones)

            for cur_phones in cur_possible_phones:
                extra_length = len(phones) - len(cur_phones)
#                 print("EL", extra_length)
                if extra_length > 0:
                    for a in range(0, extra_length + 1):
                        subphones_end = len(phones) - (extra_length - a)
                        subphones = phones[a:subphones_end]
                        if len(subphones) != len(cur_phones):
                            print("Lengths not equal....")
                        #print(subphones, cur_phones)
                        if subphones == cur_phones:
                            #print("MATCH", subword, subphones)
                            first = letterToPhone(word[0:j], phones[0:a])
                            cur = letterToPhone(subword, subphones)
                            last = letterToPhone(word[i+j:], phones[subphones_end:])
                            # print("LTP Complete", first, cur, last)
                            return first + cur + last
                    
    return [[word.upper(), phones]]

for segment in result_aligned["word_segments"]:
    word = segment["text"]
# for word in test_words:
#     print(word)
#     print(letterToPhone(word, wordbreak(word)[0]))
#     print([word, wordbreak(word)[0]], "\n")

In [64]:
chunked_words = []
for segment in result_aligned["word_segments"]:
    word = segment["text"]
    chunked_words.append(letterToPhone(word, wordbreak(word)[0]))

In [65]:
chunked_words[0]

[['G', ['G']], ['OO', ['UH']], ['D', ['D']]]

# Parsing out the transcriptions

In [66]:
ts = [o['text'] for o in result_aligned["word_segments"]]

In [67]:
ts = '|'.join(ts).upper()
ts = ts.replace(',', '')
ts = ts.replace('.', '')

In [68]:
ts

"GOOD|EVENING|EVERYONE|SYDNEY|SCOTT|HERE|BRINGING|YOU|THE|LATEST|NEWS|FROM|THE|UPSIDE|DOWN|WORLD|IT|APPEARS|THAT|PONZI|SCHEMES|ONCE|THOUGHT|TO|BE|A|FINANCIAL|FRAUD|TOOL|USED|ONLY|BY|HUMANS|HAVE|NOW|JUMPED|SPECIES|AND|KITTENS|CAN|NOW|APPARENTLY|BE|USED|TO|SCAM|PEOPLE|OUT|OF|THEIR|HARD-EARNED|MONEY|THE|SCHEME|WORKS|LIKE|ANY|OTHER|PONZI|SCHEME|WITH|THE|CATS|BEING|SOLD|FOR|A|LARGE|SUM|OF|MONEY|AND|THEN|BREEDING|MORE|CATS|TO|SELL|FOR|EVEN|MORE|ALL|TOO|OFTEN|THE|CATS|ARE|SOLD|FOR|MORE|MONEY|THAN|WHAT|THEY|ARE|ACTUALLY|WORTH|RESULTING|IN|A|HUGE|PROFIT|FOR|THE|SELLER|AND|A|HUGE|LOSS|FOR|THE|BUYER|THE|PROBLEM|IS|THAT|THESE|CATS|WHILE|CUTE|AND|CUDDLY|ARE|NOT|WORTH|THEIR|HEFTY|PRICE|TAGS|AND|ARE|NOT|EVEN|CLOSE|TO|BEING|A|TRUE|INVESTMENT|UNFORTUNATELY|THIS|NEW|FORM|OF|SCAM|HAS|BEEN|CAUSING|A|LOT|OF|FINANCIAL|ISSUES|AND|HEARTACHE|FOR|PEOPLE|IN|THE|UPSIDE-DOWN|WORLD|BUT|LUCKILY|THE|AUTHORITIES|HAVE|BEEN|CRACKING|DOWN|ON|THESE|SCHEMES|IN|AN|EFFORT|TO|PROTECT|PEOPLE'S|FINANCIAL|WELL-BEING|I'D|SUGGEST|

# FORCED ALIGNMENT FROM WAV2VEC2

In [20]:
import os
from dataclasses import dataclass

import torch
import torchaudio

import IPython
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["figure.figsize"] = [16.0, 4.8]

In [42]:
# model we are loading
MODEL_NAME = 'WAV2VEC2_ASR_BASE_LV60K_960H'
MODEL_NAME = 'WAV2VEC2_ASR_BASE_960H'
# sampling rate the model expects
# NOTE: most wav2vec models assume 16k sampling rate
MODEL_SR = 16_000


@dataclass
class Point:
    token_index: int
    time_index: int
    score: float


# Merge the labels
@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float

    def __repr__(self):
        return f"{self.label}\t({self.score:4.2f}): [{self.start:5f}, {self.end:5f})"

    @property
    def length(self):
        return self.end - self.start



def load_audio(wav_file, model_sr=MODEL_SR):
    '''Loads `wav_file` using torchaudio.
    
    Resamples the audio to `MODEL_SR` if needed.
    '''
    wav, sr = torchaudio.load(wav_file)
    if sr != model_sr:
        print(f'Resampling from {sr} to {MODEL_SR}')
        wav = torchaudio.transforms.Resample(sr, MODEL_SR)(wav)
    return wav


def clean_text(o):
    clean = o.replace(',', ' ')
    clean = clean.strip('\n')
    return clean

def load_transcript(text_file):
    '''Loads a transcript in `text_file`.
    
    Assumes one transcription per line.
    '''
    # load text file
    lines = open(text_file, encoding="utf8").readlines()
    # cleanup the text
    lines = [clean_text(line) for line in lines]
    # replace spaces with special token `|`
    lines = ['|'.join(line.split(' ')) for line in lines]
    # make all characters upper-case for wav2vec token outputs
    lines = [line.upper() for line in lines]
    return lines
    
    
def load_model(model_name):
    '''Loads a pytorch `model_name` from torchaudio.
    '''
    bundle = getattr(torchaudio.pipelines, model_name, None)
    if bundle:
        model = bundle.get_model()
    else:
        raise ValueError(f'Could not find model: {model_name}')
    return model, bundle



def get_emissions(model, audio):
    '''Gets token probabilities from `model` for the speech given in `audio`.
    '''
    emissions, _ = model(audio)
    emissions = torch.log_softmax(emissions, dim=-1)
    return emissions


def get_trellis(emission, tokens, blank_id=0):
    num_frame = emission.size(0)
    num_tokens = len(tokens)

    # Trellis has extra diemsions for both time axis and tokens.
    # The extra dim for tokens represents <SoS> (start-of-sentence)
    # The extra dim for time axis is for simplification of the code.
    trellis = torch.empty((num_frame + 1, num_tokens + 1))
    trellis[0, 0] = 0
    trellis[1:, 0] = torch.cumsum(emission[:, 0], 0)
    trellis[0, -num_tokens:] = -float("inf")
    trellis[-num_tokens:, 0] = float("inf")

    for t in range(num_frame):
        trellis[t + 1, 1:] = torch.maximum(
            # Score for staying at the same token
            trellis[t, 1:] + emission[t, blank_id],
            # Score for changing to the next token
            trellis[t, :-1] + emission[t, tokens],
        )
    return trellis


def backtrack(trellis, emission, tokens, blank_id=0):
    # Note:
    # j and t are indices for trellis, which has extra dimensions
    # for time and tokens at the beginning.
    # When referring to time frame index `T` in trellis,
    # the corresponding index in emission is `T-1`.
    # Similarly, when referring to token index `J` in trellis,
    # the corresponding index in transcript is `J-1`.
    j = trellis.size(1) - 1
    t_start = torch.argmax(trellis[:, j]).item()

    path = []
    for t in range(t_start, 0, -1):
        # 1. Figure out if the current position was stay or change
        # Note (again):
        # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
        # Score for token staying the same from time frame J-1 to T.
        stayed = trellis[t - 1, j] + emission[t - 1, blank_id]
        # Score for token changing from C-1 at T-1 to J at T.
        changed = trellis[t - 1, j - 1] + emission[t - 1, tokens[j - 1]]

        # 2. Store the path with frame-wise probability.
        prob = emission[t - 1, tokens[j - 1] if changed > stayed else 0].exp().item()
        # Return token index and time index in non-trellis coordinate.
        path.append(Point(j - 1, t - 1, prob))

        # 3. Update the token
        if changed > stayed:
            j -= 1
            if j == 0:
                break
    else:
        raise ValueError("Failed to align")
    return path[::-1]


def merge_repeats(path, transcript):
    i1, i2 = 0, 0
    segments = []
    while i1 < len(path):
        while i2 < len(path) and path[i1].token_index == path[i2].token_index:
            i2 += 1
        score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
        segments.append(
            Segment(
                transcript[path[i1].token_index],
                path[i1].time_index,
                path[i2 - 1].time_index + 1,
                score,
            )
        )
        i1 = i2
    return segments


# Merge words
def merge_words(segments, separator="|"):
    words = []
    i1, i2 = 0, 0
    while i1 < len(segments):
        if i2 >= len(segments) or segments[i2].label == separator:
            if i1 != i2:
                segs = segments[i1:i2]
                word = "".join([seg.label for seg in segs])
                score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)
                words.append(Segment(word, segments[i1].start, segments[i2 - 1].end, score))
            i1 = i2 + 1
            i2 = i1
        else:
            i2 += 1
    return words

In [43]:
# example run through


def run_forced_alignment(
    model_name,
    speech_file,
    text_file,
    device=None,
    
):
    
    # set the hardware device
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # load the model
    model, bundle = load_model(model_name)
    model.eval()
    model.to(device)

    # get model labels
    labels = bundle.get_labels()
    dictionary = {c: i for i, c in enumerate(labels)}
    print(f'Aligning wav2vec2 tokens:')
    keys = [f'{k}: {v}' for k,v in dictionary.items()]
    print('\n'.join(keys))

    # load the audio and transcript
    audio = load_audio(speech_file)
    audio = audio.to(device)
    print(f'Audio shape: {audio.shape}')
    
    # read in the transcript
    if os.path.isfile(text_file):
        transcript = load_transcript(text_file)[0]
    else:
        print(f'Note: using "{text_file}" as the transcript')
        transcript = text_file

    # get the token probabilities
    emissions = get_emissions(model, audio)
    emissions = emissions[0].detach().cpu()

    # turn the transcript into tokens
    tokens = [dictionary[c] for c in transcript]

    # populate the trellis
    trellis = get_trellis(emissions, tokens)

    # walkback to find the most likely trellis path
    path = backtrack(trellis, emissions, tokens)

    # merge the paths with repeated labels
    segments = merge_repeats(path, transcript)

    # merge the words based on the separator token '|'
    word_segments = merge_words(segments)
    
    # cleanup the model
    del model
    model = None
    torch.cuda.empty_cache()
    gc.collect()
    
    return {
        'character_segs': segments,
        'word_segs': word_segments,
    }

### Cleaning up some GPU memory

In [44]:
!nvidia-smi

Sat Mar  4 22:16:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0  On |                  Off |
| 41%   39C    P8    16W / 140W |   1490MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [45]:
del model
model = None
torch.cuda.empty_cache()
gc.collect() 

del model_a
model_a = None
torch.cuda.empty_cache()
gc.collect() 

0

In [46]:
!nvidia-smi

Sat Mar  4 22:16:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0  On |                  Off |
| 41%   39C    P8    16W / 140W |   1490MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Calling the Force Alignment on the `kitten_full.wav` file

In [48]:
speech_file = 'kitten_full.wav'
text_file = ts
model_name = MODEL_NAME


segs = run_forced_alignment(
    model_name,
    speech_file,
    text_file,
    device='cpu',
)

Aligning wav2vec2 tokens:
-: 0
|: 1
E: 2
T: 3
A: 4
O: 5
N: 6
I: 7
H: 8
S: 9
R: 10
D: 11
L: 12
U: 13
M: 14
W: 15
C: 16
F: 17
G: 18
Y: 19
P: 20
B: 21
V: 22
K: 23
': 24
X: 25
J: 26
Q: 27
Z: 28
Resampling from 22050 to 16000
Audio shape: torch.Size([1, 1271014])
Note: using "GOOD|EVENING|EVERYONE|SYDNEY|SCOTT|HERE|BRINGING|YOU|THE|LATEST|NEWS|FROM|THE|UPSIDE|DOWN|WORLD|IT|APPEARS|THAT|PONZI|SCHEMES|ONCE|THOUGHT|TO|BE|A|FINANCIAL|FRAUD|TOOL|USED|ONLY|BY|HUMANS|HAVE|NOW|JUMPED|SPECIES|AND|KITTENS|CAN|NOW|APPARENTLY|BE|USED|TO|SCAM|PEOPLE|OUT|OF|THEIR|HARD-EARNED|MONEY|THE|SCHEME|WORKS|LIKE|ANY|OTHER|PONZI|SCHEME|WITH|THE|CATS|BEING|SOLD|FOR|A|LARGE|SUM|OF|MONEY|AND|THEN|BREEDING|MORE|CATS|TO|SELL|FOR|EVEN|MORE|ALL|TOO|OFTEN|THE|CATS|ARE|SOLD|FOR|MORE|MONEY|THAN|WHAT|THEY|ARE|ACTUALLY|WORTH|RESULTING|IN|A|HUGE|PROFIT|FOR|THE|SELLER|AND|A|HUGE|LOSS|FOR|THE|BUYER|THE|PROBLEM|IS|THAT|THESE|CATS|WHILE|CUTE|AND|CUDDLY|ARE|NOT|WORTH|THEIR|HEFTY|PRICE|TAGS|AND|ARE|NOT|EVEN|CLOSE|TO|BEING|A|TRUE|INVE

In [57]:
# character segmentation map
segs['character_segs']

[G	(1.00): [5.000000, 6.000000),
 O	(0.67): [6.000000, 9.000000),
 O	(1.00): [9.000000, 10.000000),
 D	(0.52): [10.000000, 12.000000),
 |	(0.47): [12.000000, 15.000000),
 E	(0.97): [15.000000, 17.000000),
 V	(0.50): [17.000000, 19.000000),
 E	(0.50): [19.000000, 21.000000),
 N	(1.00): [21.000000, 23.000000),
 I	(1.00): [23.000000, 24.000000),
 N	(0.81): [24.000000, 26.000000),
 G	(1.00): [26.000000, 27.000000),
 |	(0.68): [27.000000, 30.000000),
 E	(1.00): [30.000000, 31.000000),
 V	(0.69): [31.000000, 33.000000),
 E	(1.00): [33.000000, 34.000000),
 R	(0.66): [34.000000, 37.000000),
 Y	(0.92): [37.000000, 45.000000),
 O	(0.97): [45.000000, 47.000000),
 N	(0.96): [47.000000, 49.000000),
 E	(0.33): [49.000000, 52.000000),
 |	(0.96): [52.000000, 76.000000),
 S	(0.87): [76.000000, 79.000000),
 Y	(0.49): [79.000000, 82.000000),
 D	(1.00): [82.000000, 84.000000),
 N	(0.50): [84.000000, 86.000000),
 E	(0.50): [86.000000, 88.000000),
 Y	(0.75): [88.000000, 92.000000),
 |	(0.66): [92.000000, 93

In [211]:
# view a few chunked words
chunked_words[2]

[['E', ['EH']],
 ['V', ['V']],
 ['E', ['IY']],
 ['RY', []],
 ['O', ['W', 'AH']],
 ['N', ['N']],
 ['E.', []]]

# HERE: MAPPING CHARACTER DURATIONS TO PHONES

In [214]:
char_segs = segs['character_segs'][:]
char_ptr = 0


def frame_to_ms(frame):
    return int(frame * 1000 / 50) # * 20

# stores the info for the needed transcriptions
full_info = []

# step through the chunked, phoneme words
for cw in chunked_words:
    
    # cleanup punctuation from the word
    word = ''.join(o[0] for o in cw).lower()
    word = word.replace(',', '')
    word = word.replace('.', '')
        
    # alignments for this word
    alns = []
    
    # step through the pairs of letters/phones for this word
    for elem in cw:
        (letters, phone) = elem
        
        # cleanup punctuation from the letters
        if letters:
            letters = letters.replace(',', '')
            letters = letters.replace('.', '')
            
        # TODO: handle cases when:
        ## A) There is no phone for the letters
        ## B) There are more than on phones for the letter
        if phone:
            phone = phone[0] # get out the element (NOTE: I was assuming there was only one phone)
                
            # group the character segments for this letter/phone pair
            cur_segs = []
            while ''.join(o.label for o in cur_segs) != letters:
                try:
                    cur_segs.append(char_segs[char_ptr])
                    char_ptr += 1
                except:
                    print(word, letters)
                    raise Exception(f'{char_ptr,len(char_segs)}')

            if cur_segs:
                alns.append({
                    'ph': phone,
                    'bg': frame_to_ms(cur_segs[0].start),
                    'ed': frame_to_ms(cur_segs[-1].end),
                })
                
    # if we hit the end of the word, advance the pointer
    if char_segs[char_ptr].label == '|':
        char_ptr += 1
    
    # add the info we need for the transcriptions
    info = {'word': word, 'phones': alns}
    full_info.append(info)


everyone O


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_6790/1443791809.py:41 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_6790/1443791809.py'                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_6790/1443791809.py:45 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_6790/1443791809.py'                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: (1286, 1286)

In [215]:
full_info

[{'word': 'good',
  'phones': [{'ph': 'G', 'bg': 100, 'ed': 120},
   {'ph': 'UH', 'bg': 120, 'ed': 200},
   {'ph': 'D', 'bg': 200, 'ed': 240}]},
 {'word': 'evening',
  'phones': [{'ph': 'IY', 'bg': 300, 'ed': 340},
   {'ph': 'V', 'bg': 340, 'ed': 420},
   {'ph': 'N', 'bg': 420, 'ed': 460},
   {'ph': 'IH', 'bg': 460, 'ed': 480},
   {'ph': 'NG', 'bg': 480, 'ed': 540}]}]

### Group into needed format

In [144]:
import pandas as pd

In [119]:
segs['character_segs'][-3]

O	(1.00): [3944.000000, 3945.000000)

In [99]:
phone_align

[{'G': {'beg': 5, 'end': 6}}]

In [ ]:
s

In [58]:
[(o.start / 50, o.end / 50) for o in segs['word_segs']]

[(0.1, 0.24),
 (0.3, 0.54),
 (0.6, 1.04),
 (1.52, 1.84),
 (1.86, 2.2),
 (2.24, 2.44),
 (2.48, 2.78),
 (2.82, 2.94),
 (3.0, 3.08),
 (3.12, 3.44),
 (3.52, 3.74),
 (3.8, 3.92),
 (3.96, 4.06),
 (4.16, 4.56),
 (4.6, 4.86),
 (4.9, 5.28),
 (6.02, 6.08),
 (6.12, 6.48),
 (6.54, 6.68),
 (6.74, 7.22),
 (7.26, 7.74),
 (8.24, 8.42),
 (8.5, 8.76),
 (8.82, 8.88),
 (8.92, 9.02),
 (9.06, 9.08),
 (9.14, 9.68),
 (9.78, 10.16),
 (10.24, 10.66),
 (10.82, 11.0),
 (11.12, 11.34),
 (11.38, 11.54),
 (11.62, 12.1),
 (12.58, 12.72),
 (12.78, 13.02),
 (13.12, 13.38),
 (13.48, 14.1),
 (14.56, 14.66),
 (14.7, 15.04),
 (15.1, 15.22),
 (15.28, 15.42),
 (15.5, 16.1),
 (16.16, 16.28),
 (16.42, 16.64),
 (16.76, 16.86),
 (16.98, 17.3),
 (17.34, 17.7),
 (17.84, 17.94),
 (17.98, 18.02),
 (18.04, 18.26),
 (18.32, 18.94),
 (19.02, 19.28),
 (20.18, 20.28),
 (20.32, 20.58),
 (20.62, 20.88),
 (20.96, 21.14),
 (21.24, 21.38),
 (21.46, 21.66),
 (21.7, 22.14),
 (22.16, 22.48),
 (22.5, 22.62),
 (22.64, 22.7),
 (22.76, 23.06),
 (23.

# Comparing the alignments

In [129]:
result_aligned["word_segments"];

### NODE: older code below

In [ ]:
# {'text': 'Good', 'start': 0.10120481927710842, 'end': 0.32385542168674697},
# INTO
# [
#     {
#         "word": "sil",
#         "phones": [
#             {
#                 "ph": "SIL",
#                 "bg": 0,
#                 "ed": 126
#             }
#         ]
#     },
#     {
#         "word": "YOU",
#         "phones": [
#             {
#                 "ph": "Y",
#                 "bg": 126,
#                 "ed": 138
#             },
#             {
#                 "ph": "UW",
#                 "bg": 138,
#                 "ed": 141
#             }
#         ]
#     },
from math import trunc

def makeSegment(word, phones, start, end):
    total = end - start
    time_per_phone = trunc(total / len(phones))
#     print(total, time_per_phone)
    expanded_phones = [{"ph": phone, "bg": start + (i * time_per_phone), "ed": start + ((i + 1) * time_per_phone)} for i, phone in enumerate(phones)]
    return {"word": word, "phones": expanded_phones}

def makeSil(start, end):
    return {"word": "sil", "phones": [{"ph": "SIL", "bg": start, "ed": end}]}
  
all_phonemes = []
for segment in result_aligned["word_segments"]:
    word = segment["text"]
    #Get rid of numbers in phones
    phonemes = wordbreak(word)[0]
    print(phonemes)
    start_time = trunc(segment["start"] * 100)
    end_time = trunc(segment["end"] * 100)
    
    if len(all_phonemes) > 0:
        last = all_phonemes[-1]
        last_end = last["phones"][-1]["ed"]
        if start_time - last_end > 5:
            sil = makeSil(last_end, start_time)
            all_phonemes.append(sil)
        else:
            start_time = last_end
    else:
        sil = makeSil(0, start_time)
        all_phonemes.append(sil)
        
    segment = makeSegment(word, phonemes, start_time, end_time)
    all_phonemes.append(segment)

all_phonemes

In [ ]:
# import json

# file_name = "../samples/phonemes/{name}.json".format(name=audio_file_name)
# print("Writing to ", file_name)
# f1 = open(file_name, 'w')
# json.dump(all_phonemes, f1)

In [ ]:
# all_phonemes